<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!wget https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv

--2020-04-09 20:54:04--  https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4148456 (4.0M) [text/plain]
Saving to: ‘complaints.csv’

complaints.csv      100%[===================>]   3.96M  19.9MB/s    in 0.2s    

2020-04-09 20:54:06 (19.9 MB/s) - ‘complaints.csv’ saved [4148456/4148456]



In [14]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 798kB/s 
     |████████████████████████████████| 7.1MB 3.3MB/s 
     |████████████████████████████████| 378kB 27.5MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=859603 sha256=8d6f141be5f48d249a13e4c25cfabe857cd3ff0718e32f92af94d3cec079110d
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [0]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression, LinearRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier,  BaggingClassifier, BaggingRegressor, RandomTreesEmbedding,GradientBoostingClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk import word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [61]:
data = pd.read_csv('complaints.csv', sep='\t')
print(data.head(10))
y = data["PRODUCT_ID"]
X = data["cleaned_text"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

   COMPLAINT_ID  ...                                       cleaned_text
0       3178905  ...  go year . contact advis never took loan . advi...
1       3175952  ...  mail valid debt xx/xx/19 valid receiv , receiv...
2       3174747  ...  xx/xx/xxxx appli receiv onlin loan bluechip fi...
3       3173291  ...  xx/xx/xxxx appli receiv onlin loan . loan amou...
4       3172221  ...  told husband left bill . debt would pay within...
5       3172723  ...  payday loan , becam abl pay . everyon famili c...
6       3171497  ...  within past week , contact oracl financi group...
7       3171674  ...  call work said courthous owe money . got phone...
8       3170784  ...  today xx/xx/xxxx cst contact ks . receiv appro...
9       3171479  ...  cashcal attempt collect fraudul debt . ident s...

[10 rows x 5 columns]


In [0]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import FunctionTransformer

In [29]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
clf1 = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=3)
clf2 = RandomForestClassifier(n_estimators=250, random_state=3)
clf3 = GaussianNB()
clf4 = SVC(class_weight="balanced", random_state =30)
eclf = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3), ('svc', clf4)], voting='hard')

voting = Pipeline([
    ('vect', CountVectorizer( analyzer='word', max_features=360, ngram_range=(1,1))),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
    ('reducer', TruncatedSVD(n_components=120)),
    ('clf', eclf),
    ])
voting = voting.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [31]:
predictions = voting.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

Precision:   0.70
Recall:   0.69
F1-measure:   0.69
Accuracy:   0.69


Почти 70, посмотрим теперь bagging.

In [0]:
%matplotlib inline

import itertools
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier

In [0]:
clf1 = DecisionTreeClassifier(criterion='entropy', max_depth=1)
clf2 = KNeighborsClassifier(n_neighbors=1)    

bagging1 = BaggingClassifier(base_estimator=clf1, n_estimators=20, max_samples=0.8, max_features=0.8)
bagging2 = BaggingClassifier(base_estimator=clf2, n_estimators=20, max_samples=0.8, max_features=0.8)

In [0]:
bag = Pipeline([
    ('vect', CountVectorizer( analyzer='word', max_features=360, ngram_range=(1,1))),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
    ('reducer', TruncatedSVD(n_components=120)),
    ('clf', bagging1),
    ])
bag = bag.fit(X_train, y_train)

In [24]:
predictions = bag.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

Precision:   0.20
Recall:   0.36
F1-measure:   0.21
Accuracy:   0.37


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
bag2 = Pipeline([
    ('vect', CountVectorizer( analyzer='word', max_features=360, ngram_range=(1,1))),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
    ('reducer', TruncatedSVD(n_components=200)),
    ('clf', bagging2),
    ])
bag2 = bag2.fit(X_train, y_train)

In [26]:
predictions = bag2.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

Precision:   0.63
Recall:   0.63
F1-measure:   0.63
Accuracy:   0.63


In [0]:
#Не очень много получилось, но может я не так что-то сделал. Теперь Boosting.

In [0]:
from sklearn.ensemble import AdaBoostClassifier

In [0]:
adb_clf = DecisionTreeClassifier(criterion='entropy', max_depth=10)

In [0]:
num_est = [1, 2, 3, 10]
label = ['AdaBoost (n_est=1)', 'AdaBoost (n_est=2)', 'AdaBoost (n_est=3)', 'AdaBoost (n_est=10)']

In [0]:

boost = AdaBoostClassifier(base_estimator=adb_clf, n_estimators=n_est)
boosting = Pipeline([
('vect', CountVectorizer( analyzer='word', max_features=350, ngram_range=(1,1))),
('tfidf', TfidfTransformer(sublinear_tf=True)),
('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
('reducer', TruncatedSVD(n_components=200)),
('clf', boost),
])   
boosting = boosting.fit(X_train, y_train)

In [114]:
predictions = boosting.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

Precision:   0.53
Recall:   0.54
F1-measure:   0.53
Accuracy:   0.53


In [0]:
#Тоже плохо, и теперь Stacking.

In [0]:
from mlxtend.classifier import StackingClassifier

In [0]:
clf1 = KNeighborsClassifier(n_neighbors=1)
clf3 = GaussianNB()
clf2 = RandomForestClassifier(n_estimators=200, random_state=1)
clf4 = LogisticRegression(multi_class='multinomial', random_state=1)
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3, clf4], 
                          meta_classifier=lr)

In [133]:
stacking_pipe = Pipeline([
('vect', CountVectorizer(analyzer='word', max_features=300, ngram_range=(1,1))),
('tfidf', TfidfTransformer(sublinear_tf=True)),
('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
('reducer', TruncatedSVD(n_components=200)),
('clf', sclf),
])   
stacking = stacking_pipe.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [134]:
predictions = stacking.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

Precision:   0.37
Recall:   0.43
F1-measure:   0.38
Accuracy:   0.42


In [0]:
clf1 = KNeighborsClassifier(n_neighbors=1, leaf_size=35)
clf2 = RandomForestClassifier(random_state=1, min_samples_leaf=1)
clf3 = GaussianNB()
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=lr)

In [168]:
stacking_pipe = Pipeline([
('vect', CountVectorizer(analyzer='word', max_features=350, ngram_range=(1,1))),
('tfidf', TfidfTransformer(sublinear_tf=True)),
('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
('reducer', TruncatedSVD(n_components=300)),
('clf', sclf),
])   
stacking = stacking_pipe.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [169]:
predictions = stacking.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

Precision:   0.58
Recall:   0.57
F1-measure:   0.57
Accuracy:   0.57


In [0]:
#Тоже не очень, хотя вроде я попробовал разные параметры.Получается, что лучше всех сработал VotingClassifier.